Compare different calculation methods for BAHAMAS, generally to improve log likelihood calculation speed while ensuring results stay the same. 

In [1]:
# add path to bahamas code to PATH
import sys
sys.path.append('..')

In [2]:
import pandas as pd
import numpy as np
import bahamas, priors, time

In [3]:
# load data 
sim_number = sys.argv[1] if len(sys.argv) == 2 else 1 # default to dataset 1
data = pd.read_csv('/home/et3918/mlacceleration/SNe_samples/selected_{}.txt'.format(sim_number), sep='\s+')
data = np.array(data[['zCMB', 'zHD', 'c', 'x1', 'mB']])

sigmaC = np.array(pd.read_csv('/home/et3918/mlacceleration/SNe_samples/stats_sel_{}.txt'.format(sim_number), sep='\s+', header=None))

log_sigmaCinv = (-1 * np.linalg.slogdet(sigmaC)[1])
sigmaCinv = np.linalg.inv(sigmaC)

ndat = len(data)

J = []
for i in range(ndat):
    J.append([1., 0., 0.])
    J.append([0., 1., 0.])
    J.append([0., 0., 1.])
J  = np.matrix(J)

# comparison at single theta

In [4]:
param = [.14, 3.2, np.exp(.560333), np.exp(-2.3171), .1, -0.000320865, 0.011789294, -19.365, .3, .7, .7]

In [5]:
%time bahamas.vanilla_log_likelihood(J, sigmaCinv, log_sigmaCinv, param, data, ndat)

CPU times: user 1.88 s, sys: 222 ms, total: 2.1 s
Wall time: 178 ms


-445.32394559170064

In [6]:
%time bahamas.rubin_log_likelihood(J, sigmaCinv, log_sigmaCinv, param, data, ndat)

CPU times: user 3.65 s, sys: 375 ms, total: 4.02 s
Wall time: 336 ms


-126.34629699614555

In [7]:
%time bahamas.vincent_log_likelihood(J, sigmaCinv, log_sigmaCinv, param, data, ndat)

CPU times: user 1.85 s, sys: 226 ms, total: 2.08 s
Wall time: 248 ms


17.045378289307052

# comparison at randomly chosen theta

In [8]:
def theta_sampler(cube):
    cube[0] = cube[0] * 1
    cube[1] = cube[1] * 4
    cube[2] = priors.log_uniform(cube[2], 10**-5, 10**2)
    cube[3] = priors.log_uniform(cube[3], 10**-5, 10**2)
    cube[4] = cube[4] * 1 
    cube[5] = priors.gaussian(cube[5], 0.0, 1.)
    cube[6] = priors.gaussian(cube[6], 0.0, 10.)
    cube[7] = priors.gaussian(cube[7], -19.3, 2.)
    cube[8] = cube[8] * 1
    cube[9] = cube[9] * 2
    cube[10] = 0.3 + cube[10] * 0.7
    return cube

In [13]:
log_likelihood_times  = []
rubin_log_likelihood_times = []
vincent_log_likelihood_times = []
for i in range(100):
    random_param = theta_sampler(np.random.rand(11))
    
    a = time.time()
    vanilla = bahamas.vanilla_log_likelihood(J, sigmaCinv, log_sigmaCinv, random_param, data, ndat)
    b = time.time()
    rubin = bahamas.rubin_log_likelihood(J, sigmaCinv, log_sigmaCinv, random_param, data, ndat)
    c = time.time()
    vincent = bahamas.vincent_log_likelihood(J, sigmaCinv, log_sigmaCinv, random_param, data, ndat)
    d = time.time()
    
    log_likelihood_times.append(b - a)
    rubin_log_likelihood_times.append(c - b)
    vincent_log_likelihood_times.append(d - c)
    
    print('vanilla: {} ({}) \t rubin: {} ({}) \t vincent: {} ({}) \t diff: {}'.format(
        round(vanilla, 3),
        round(b - a, 3),
        round(rubin, 3),
        round(c - b, 3),
        round(vincent, 3),
        round(d - c, 3),
        round(vincent - rubin, 3)))

vanilla: -25258.738 (0.175) 	 rubin: -24908.479 (0.343) 	 vincent: -24907.54 (0.211) 	 diff: 0.938
vanilla: -83789.33 (0.157) 	 rubin: -83789.041 (0.318) 	 vincent: -83788.224 (0.253) 	 diff: 0.817
vanilla: -147253.945 (0.189) 	 rubin: -146641.851 (0.341) 	 vincent: -146591.902 (0.258) 	 diff: 49.949
vanilla: -64988.756 (0.158) 	 rubin: -64620.699 (0.278) 	 vincent: -64611.255 (0.253) 	 diff: 9.443
vanilla: -3203086.279 (0.192) 	 rubin: -3106949.81 (0.276) 	 vincent: -3193722.492 (0.183) 	 diff: -86772.682
vanilla: -792208.782 (0.16) 	 rubin: -791855.818 (0.282) 	 vincent: -791713.137 (0.229) 	 diff: 142.682
vanilla: -360623.16 (0.163) 	 rubin: -360610.454 (0.28) 	 vincent: -360603.457 (0.229) 	 diff: 6.997
vanilla: -3365803.542 (0.161) 	 rubin: -3336015.457 (0.282) 	 vincent: -3359969.471 (0.258) 	 diff: -23954.014
vanilla: -421494.121 (0.158) 	 rubin: -421177.417 (0.266) 	 vincent: -421168.86 (0.218) 	 diff: 8.557
vanilla: -14658.131 (0.155) 	 rubin: -14658.131 (0.279) 	 vincent: -14

../bahamas/cosmology.py:29: RuntimeWarning: invalid value encountered in sqrt
  omegam*(1+zba)**3 + omegade*(1+zba)**(3.+3.*w) + (1.-omegam-omegade)*(1.+zba)**2


vanilla: -161591.377 (0.166) 	 rubin: -161391.659 (0.31) 	 vincent: -161272.327 (0.262) 	 diff: 119.332
vanilla: -14043.516 (0.174) 	 rubin: -13523.516 (0.33) 	 vincent: -13502.727 (0.224) 	 diff: 20.789
vanilla: -1e+90 (0.16) 	 rubin: -1e+90 (0.267) 	 vincent: -1e+90 (0.221) 	 diff: 0.0
vanilla: -80724.667 (0.16) 	 rubin: -80314.762 (0.276) 	 vincent: -80237.785 (0.27) 	 diff: 76.977
vanilla: -256345.457 (0.181) 	 rubin: -246637.243 (0.343) 	 vincent: -252268.15 (0.279) 	 diff: -5630.907
vanilla: -9958454.529 (0.175) 	 rubin: -9958083.724 (0.343) 	 vincent: -9957945.1 (0.251) 	 diff: 138.625
vanilla: -48624.156 (0.178) 	 rubin: -48604.902 (0.34) 	 vincent: -48588.988 (0.25) 	 diff: 15.914
vanilla: -8466.563 (0.175) 	 rubin: -8113.806 (0.339) 	 vincent: -8112.317 (0.222) 	 diff: 1.489
vanilla: -14066763.847 (0.171) 	 rubin: -14066763.847 (0.295) 	 vincent: -14066763.846 (0.242) 	 diff: 0.001
vanilla: -24336940.677 (0.169) 	 rubin: -24336940.677 (0.336) 	 vincent: -24336940.675 (0.232) 

In [14]:
log_likelihood_times = np.array(log_likelihood_times)
rubin_log_likelihood_times = np.array(rubin_log_likelihood_times)
vincent_log_likelihood_times = np.array(vincent_log_likelihood_times)

In [15]:
print(np.mean(log_likelihood_times), 
      np.mean(rubin_log_likelihood_times), 
      np.mean(vincent_log_likelihood_times))

0.15859220743179323 0.2862197279930115 0.2257496500015259
